### 학습목표
1. 로그데이터로 고객 이탈 페이지 확인하기

In [ ]:
import pandas as pd
import numpy as np

#### 웹서버 로그 데이터
 - 웹서버에 클라이언트로의 요청(request) 전달 시, 해당 요청에 대한 정보(ip, 시각, 방문 페이지 등등)를 기록하는 파일
 - 기록되는 로그의 포맷(format)의 표준이 있으나 설정으로 포맷 변경 가능
 - 로그 데이터는 주로 웹 서버의 디버깅, 데이터 분석 등의 형태로 사용 됨
 - 예제에서 사용하는 형식
   - ip 세션아이디 사용자식별자 시각 요청 페이지 상태코드 바이트사이즈
   ```
   1.0.0.1 sessionid user59 [16/Dec/2019:02:00:08] GET /checkout 200 1508
   ```

In [ ]:
logs = pd.read_csv('web.log', 
                   sep='\s',
                   engine='python',
                   names=['ip', 'session_id', 'user_id', 'datetime', 'request', 'url', 'status', 'bytesize'])

logs.head()

In [ ]:
logs.info()

In [ ]:
logs.describe()

#### 날짜 형식 변환

In [ ]:
# 01/Dec/2019T00:47:11
logs['datetime'] = logs['datetime'].apply(lambda date: date.replace('[', '').replace(']', ''))
logs['datetime'] = pd.to_datetime(logs['datetime'], format='%d/%b/%YT%H:%M:%S')

logs.head()

In [ ]:
logs.info()

#### 어떤 페이지에서 고객이 이탈을 할까?
 - 고객 이탈 페이지를 알면 해당 페이지를 분석하여 고객을 최종 단계로 더 많이 유도 가능
 - 대부분의 경우 다음 스텝으로 넘어갈때의 장벽이(신용카드 입력, 정보 입력 등등) 높은 경우가 해당 됨

In [ ]:
logs.head()

In [ ]:
session_id, product_list, product_detail, cart, order_complete
1           XXXX          YYYY            N     N

#### 퍼널 스텝 dataframe 생성
 - 스텝 순서(ordering) 등을 명시하기 위해 사용

In [ ]:
funnel_dict = {'/product_list' : 1, '/product_detail': 2, '/cart': 3, '/order_complete': 4}
funnel_steps = pd.DataFrame.from_dict(funnel_dict, orient='index', columns=['step_no'])
funnel_steps

#### session, url 로 grouping
 - user_id가 아닌 session을 기준으로 삼는 이유는 동일한 유저가 다른 세션으로 접속한 경우도 다른 경우로 간주해야 하기 때문
 - session_id와 url로 그루핑하여 가장 시간대가 빠른 해당 이벤트에 대해 추출

In [ ]:
grouped = logs.groupby(['session_id', 'url'])['datetime'].agg(np.min)
grouped = pd.DataFrame(grouped).merge(funnel_steps, left_on='url', right_index=True)

grouped.tail()

#### 퍼널 테이블 생성
 - 각 퍼널의 스텝이 순서대로 columns으로 오도록 변경

In [ ]:
funnel = grouped.reset_index().pivot(index='session_id', columns='step_no', values='datetime')
funnel.columns = funnel_steps.index
funnel.head()

#### 퍼널 카운트 계산
 - 각 퍼널 스텝별 카운트 계산

In [ ]:
step_values = [funnel[index].notnull().sum() for index in funnel_steps.index]
step_values

In [ ]:
def show_funnel(funnel_values, funnel_steps):
    from plotly import graph_objects as go
    
    fig = go.Figure(go.Funnel(
        y = funnel_steps,
        x = funnel_values
    ))
    
    fig.show()

In [ ]:
show_funnel(step_values, funnel_steps.index)

#### 평균 시간 계산
 - 각 퍼널별 소요 시간 계산

In [ ]:
np.mean(funnel['/product_detail'] - funnel['/product_list'])

In [ ]:
np.mean(funnel['/cart'] - funnel['/product_detail'])

In [ ]:
np.mean(funnel['/order_complete'] - funnel['/cart'])